In [1]:
] activate ../ 

In [2]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

using PedestrianAvoidancePOMDP
using EmergencyBrakingSystem

┌ Info: Recompiling stale cache file /home/xubuntu/.julia/compiled/v1.0/PedestrianAvoidancePOMDP/UuJ2S.ji for PedestrianAvoidancePOMDP [a80ff0fe-e6a0-11e8-3b05-15413bd5ea7a]
└ @ Base loading.jl:1187
┌ Warning: Package PedestrianAvoidancePOMDP does not have EmergencyBrakingSystem in its dependencies:
│ - If you have PedestrianAvoidancePOMDP checked out for development and have
│   added EmergencyBrakingSystem as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PedestrianAvoidancePOMDP
└ Loading EmergencyBrakingSystem into PedestrianAvoidancePOMDP from project dependency, future warnings for PedestrianAvoidancePOMDP are suppressed.


In [8]:


#algorithm = "EmergencyBrakingSystem"
algorithm = "PedestrianAvoidancePOMDP"
#algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy = load("../policy/policy.jld2")["policy"];

# Definition ego vehicle and pedestrian behavior
scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]


ego_v = vut_speeds[9]
hit_point = hit_points[1]
scenario = scenarios[1]


ego_vehicle = []
ego_a = [] 
collision = []
belief = []
action_pomdp = []
collision_rate = []
ttc = []
risk = []
emergency_brake_request = []
prediction_obstacle = []

### Simulate scenario with parameters above defined
# generate scenario based on scenario type
(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
println(ego_x, " ", ego_y, " ", ego_v, " ", ped_x," ",  ped_y," ",  ped_v)

# simulate scenario
if (algorithm == "EmergencyBrakingSystem")
    (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
else
    (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
end
#evaluate result
(collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)



56.33333333333333 0.0 13.88888888888889 100.0 -5.066666666666666 1.3888888888888888
PedestrianAvoidancePOMDP


(false, false, 0.0, 8.082638888888901, -3.6029411764705883, 43.0, -4.0)

In [7]:
# Visualize scenario
cam = StaticCamera(VecE2(95.0,0.0),15.0)
#cam = CarFollowCamera(1,10.0)

if (algorithm == "EmergencyBrakingSystem")
    duration, fps, render_hist = EmergencyBrakingSystem.animate_record(rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, cam)
else
    duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, prediction_obstacle, cam)
end

film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video


Frames{MIME{Symbol("image/png")}}("/tmp/tmpeXDg4i", 0x00000000000000a1, 20.0, nothing)

In [5]:



# Evaluate EuroNCAP scenarios

using CSV
using DataFrames



algorithm = "EmergancyBrakingSystem"
#algorithm = "PedestrianAvoidancePOMDP"
#algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy_name = "no"
#policy_name = "longitudinal_support"
#policy_name = "longitudinal_lateral_support"
policy = load("../policy/policy.jld2")["policy"];


log_filename = string("results_", algorithm, "_", policy_name, ".csv")

println(log_filename)

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
#vut_speeds = [50.]
vut_speeds = vut_speeds / 3.6


println("Algorithm to evaluate: ", algorithm, " Policy: ", policy_name, " (if specified)")

# Evaluate all EuroNCAP scenarios
rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, collision, ego_a
results = Vector[]
for scenario in scenarios
    if (scenario == "FP") 
        hit_points = [-100., 200.]
    else
        hit_points = [0., 10., 20., 30., 40., 50.]
    end
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
            if algorithm == "EmergancyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            elseif algorithm == "PedestrianAvoidancePOMDP"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
            elseif algorithm == "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
            else
                println("No valid algorithm defined!")
                return false
            end
            println(scenario, " HP: ", hit_point, " ", ego_x, " ", ego_y, " ", ego_v, " ", ped_x," ",  ped_y," ",  ped_v)
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)
            println("Collision: ", collision, " eb: ", emergency_brake_intervention, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)

CSV.write(log_filename, df);

(sum_collisions, sum_eb, dv, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenariosMetric(results)

println("#collisions: ", sum_collisions, " #emergency brake interventions: ", sum_eb, " dv_m: ", dv, " v_mean: ", v_mean, " a_mean: ", a_mean, " a_jerk: ", a_jerk, " a_min: ", a_min)

df


results_EmergancyBrakingSystem_no.csv
Algorithm to evaluate: EmergancyBrakingSystem Policy: no (if specified)
CPCN HP: 0.0 56.33333333333333 0.0 13.88888888888889 100.0 -5.066666666666666 1.3888888888888888
Collision: true eb: true 8.63888888888889 13.474954462659397 -8.846153846153847 10.0 -10.0
CPCN HP: 10.0 56.33333333333333 0.0 13.88888888888889 100.0 -4.886666666666666 1.3888888888888888
Collision: true eb: true 5.138888888888889 12.884982638888905 -9.25 10.0 -10.0
CPCN HP: 20.0 56.33333333333333 0.0 13.88888888888889 100.0 -4.706666666666666 1.3888888888888888
Collision: true eb: true 6.638888888888889 13.107142857142874 -9.117647058823529 10.0 -10.0
CPCN HP: 30.0 56.33333333333333 0.0 13.88888888888889 100.0 -4.526666666666666 1.3888888888888888
Collision: false eb: true 0.0 11.968599033816442 -9.5 20.0 -10.0
CPCN HP: 40.0 56.33333333333333 0.0 13.88888888888889 100.0 -4.346666666666666 1.3888888888888888
Collision: false eb: true 0.0 11.968599033816442 -9.5 20.0 -10.0
CPCN HP: 

,scenario_id,ego_v,hit_point,collision,eb_intervention,dv_collision,v_mean,a_mean,a_jerk,a_min
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,13.8889,0.0,1.0,1.0,8.63889,13.475,-8.84615,10.0,-10.0
2,1.0,13.8889,10.0,1.0,1.0,5.13889,12.885,-9.25,10.0,-10.0
3,1.0,13.8889,20.0,1.0,1.0,6.63889,13.1071,-9.11765,10.0,-10.0
4,1.0,13.8889,30.0,0.0,1.0,0.0,11.9686,-9.5,20.0,-10.0
5,1.0,13.8889,40.0,0.0,1.0,0.0,11.9686,-9.5,20.0,-10.0
6,1.0,13.8889,50.0,0.0,1.0,0.0,11.2005,-9.5,20.0,-10.0
7,2.0,13.8889,0.0,0.0,1.0,0.0,11.9686,-9.5,20.0,-10.0
8,2.0,13.8889,10.0,0.0,1.0,0.0,11.2005,-9.5,20.0,-10.0
9,2.0,13.8889,20.0,0.0,1.0,0.0,11.9686,-9.5,20.0,-10.0
